In [ ]:
from pint import UnitRegistry

import numpy as np
import tabulate
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt

In [ ]:
# Инициируем работу с единицами измерения
ureg = UnitRegistry()
Q = ureg.Quantity


ureg.define("person=people")
Person = People = Q(1, "person")

In [ ]:
# Входные параметры:
# 1. Жилая площадь
living_area = 70*ureg.meter**2
ceiling_height = 2.6*ureg.meter

# 2. Количество людей
people_count = 3*People

# 3. Температура
street_temp = Q(-40, ureg.celsius)
internal_temp = Q(24, ureg.celsius)
supply_air_temp = Q(24, ureg.celsius)

# 4. Параметры воздухообмена
required_air_per_person = 30 * ureg.meter**3/(Person*ureg.hour)
living_space_airflow_k = 1 * 1/ureg.hour

# 5. Параметры влажности воздуха
required_humidity = 40*ureg.percent
# 6. Начальная температура воды
water_starting_temp = Q(10, ureg.celsius)

In [ ]:
# Табличные величины
# Воздух
air_specific_heat = 1030.00 * ureg.joule/(ureg.kg*ureg.kelvin)
air_mole_weight = 29*ureg.gram/ureg.mole
air_mole_volume = 22.4*ureg.liter/ureg.mole
air_density = (air_mole_weight/air_mole_volume).to(ureg.kg/ureg.meter**3)

air_specific_heat_vol = air_specific_heat*air_density

# Вода
# Удельная теплоёмкость
water_specific_heat = 4180.6 * ureg.joule/(ureg.kg*ureg.kelvin)
# Удельная теплота парообразования
water_specific_heat_vpr = 2260000 * ureg.joule/ureg.kg

tabulate.tabulate(
    tablefmt='html',
    tabular_data=[
        ["Плотность воздуха", air_density],
        ["Объёмная удельная теплоёмкость воздуха", air_specific_heat_vol],
    ]
)

In [ ]:
# Таблица влажности воздуха

abs_hum_unit = ureg.gram/ureg.meter**3
abs_hum_unit_metric = ureg.kg/ureg.meter**3

air_table_temperatures = [Q(x, ureg.celsius).to(ureg.kelvin) for x in [
    -50, -40, -30, -20, -15, -10, -5,
    0,
    5, 10, 15, 20, 30, 35,
]]
air_table_abs_humidies = [x*abs_hum_unit for x in [
    0.063, 0.177, 0.455, 1.076, 1.608, 2.361, 3.411,
    4.852,
    6.801, 9.403, 12.83, 17.3, 30.36, 39.58,
]]

air_table_temperatures_untyped = Q.from_list(air_table_temperatures)
air_table_abs_humidies_untyped = Q.from_list(air_table_abs_humidies)

air_table_linear = interpolate.interp1d(air_table_temperatures_untyped, air_table_abs_humidies_untyped)
# air_table_linear2 = interpolate.Akima1DInterpolator(air_table_temperatures_untyped, air_table_abs_humidies_untyped)
air_table_spline = interpolate.CubicSpline(air_table_temperatures_untyped, air_table_abs_humidies_untyped)

interpolate_fn = air_table_spline

def air_max_humidity(temp: float) -> float:
    return (interpolate_fn(temp.to(ureg.kelvin)/ureg.kelvin)*abs_hum_unit).to(abs_hum_unit_metric)

temps = np.linspace(min(air_table_temperatures_untyped), max(air_table_temperatures_untyped), 1000)


# air_max_humidity(Q(22, ureg.celsius))
plt.plot(
    Q.from_list(air_table_temperatures_untyped, ureg.celsius),
    air_table_abs_humidies_untyped,
    'o'
)
plt.plot(
    Q.from_list(temps, ureg.celsius),
    air_table_spline(temps),
    '-',
)
# plt.plot(
#     Q.from_list(temps, ureg.celsius),
#     air_table_linear(temps),
#     '-',
# )
# plt.plot(
#     Q.from_list(temps, ureg.celsius),
#     air_table_linear2(temps),
#     '-',
# )
plt.show()

In [ ]:
living_volume = (living_area*ceiling_height).to(ureg.meter**3)
living_volume

In [ ]:
required_airflow_by_people = required_air_per_person*people_count

required_airflow_by_living_volume = living_volume*living_space_airflow_k
required_airflow = max(required_airflow_by_people, required_airflow_by_living_volume)



tabulate.tabulate(
    tablefmt='html',
    tabular_data=[
        ["Жилой объём", living_volume],
        ["Приток по людям", required_airflow_by_people],
        ["Приток по воздухообмену", required_airflow_by_living_volume],
        ["Необходимый приток", required_airflow],
        # ["Точка росы", dew_point],
        # ["Необходимое количество воды для увлажнения приточного воздуха", required_water_volume_per_hour]
    ]
)

In [ ]:
# Энергетика вентиляции
# 1. Нагрев воздуха
def air_heat_power(required_airflow, street_temp, supply_air_temp) -> float:
    return (required_airflow*air_specific_heat_vol*(supply_air_temp-street_temp).to(ureg.kelvin)).to(ureg.kilowatt)



# 2. Увлажнение воды
def humidification_power(required_airflow, required_humidity, street_temp, street_humidity) -> float:
    dew_street_point = air_max_humidity(street_temp)
    dew_point = air_max_humidity(internal_temp)

    required_water_volume_per_hour = required_airflow * (dew_point * float(required_humidity) - dew_street_point * float(street_humidity))

    evaporation_energy = (Q(100, ureg.celsius) - water_starting_temp).to(ureg.kelvin)*water_specific_heat + water_specific_heat_vpr

    return (required_water_volume_per_hour*evaporation_energy).to(ureg.kilowatt)

air_heat_pwr = air_heat_power(required_airflow, street_temp, supply_air_temp)
humidification_pwr = humidification_power(required_airflow, required_humidity, street_temp, required_humidity)

tabulate.tabulate(
    tablefmt='html',
    headers=["Результат"],
    tabular_data=[
        ["Мощность на нагрев воздуха", air_heat_pwr],
        ["Мощность на увлажнение", humidification_pwr],
        ["Суммарная мощность", sum([air_heat_pwr, humidification_pwr])],
    ]
)



In [ ]:
temperatures = np.linspace(-40, 20, 200)

air_heat_pwrs = [
    air_heat_power(required_airflow, Q(street_temp, ureg.celsius), supply_air_temp)
    for street_temp in temperatures
]

# Для простоты считаем что относительная влажность на улице равна относительной
# влажности необходимой нам в приточном воздухе
humidification_pwrs = [
    humidification_power(required_airflow, required_humidity, Q(street_temp, ureg.celsius), required_humidity)
    for street_temp in temperatures
]

summary_pwrs = [
    air_heat_power(required_airflow, Q(street_temp, ureg.celsius), supply_air_temp) +
    humidification_power(required_airflow, required_humidity, Q(street_temp, ureg.celsius), required_humidity)
    for street_temp in temperatures
]


plt.plot(
    temperatures,
    Q.from_list(air_heat_pwrs),
    '-',
    label="Air Heat Power",
)
plt.plot(
    temperatures,
    Q.from_list(humidification_pwrs),
    '-',
    label="Humidification Power",
)
plt.plot(
    temperatures,
    Q.from_list(summary_pwrs),
    '-',
    label="Summary power",
)
plt.xlabel("Street Temp [deg C]")
plt.ylabel("Required Vent Power [J]")
plt.legend()
plt.show()